# Lecture 2 Development Notebook for Serving Models in Streamlit

In [ ]:
## Imports
import joblib
from lime.lime_tabular import LimeTabularExplainer

## Setting Filepath (If not provided)

In [ ]:
from pprint import pprint

FPATHS = dict(
    demo_data={
        "raw": {
            "full": "Data/loan_approval.csv"
        },
        "ml": {
            "train": "Data/train.joblib",  
            "test": "Data/test.joblib", 
        },
        "nlp": {
            "nlp_full": "data/nlp/nlp_full.csv"} # raw movie review data 
            
    },
    demo_models={
        "linear_regression": "models/logreg_pipe.joblib", 
        "random_forest": "models/rf_pipe.joblib", 
    },
    demo_images={
        "banner": "Images/money_tree.png", 
    },
)
pprint(FPATHS)


In [ ]:
# save this file path in config folder to use it for deployment

import os, json
os.makedirs('config/', exist_ok=True)
FPATHS_FILE = 'config/filepaths.json'
with open(FPATHS_FILE, 'w') as f:
    json.dump(FPATHS, f)



In [ ]:
# using custom function create the dictionaries using this path

# first need to import autorelaod function 
%load_ext autoreload
%autoreload 2
import function as fn

We need to append the absolute filepath of the folder that contains our module. (the folder up one level "../")

In [ ]:
# Add the Module to the Python Path

import sys, os
# Check sys.path for python path
sys.path

In [ ]:
# Get the absolute file path of parent directory
os.path.abspath('../')

In [ ]:
# Add parent directory to python path
sys.path.append( os.path.abspath('../'))
sys.path[-1]

In [ ]:

# Try importing again
import functions as fn

In [ ]:
fn.create_directories_from_paths(FPATHS)

In [ ]:
# Lets checck fpath

with open('config/filepaths.json', 'r') as f:
    path = json.load(f)

In [ ]:
path.keys()

In [ ]:
path['demo_data']

In [ ]:
path['demo_data']['nlp']

## Using the filepath alaready set 

In [ ]:
## Open 

In [ ]:
## Import pathlib file
pathlib = joblib.load('pathlib.joblib')
pathlib

In [ ]:
# def load_data(pathlib) to load data from pathlib file
def load_data(pathlib):
    train_path = pathlib['data']['train']
    X_train, y_train =  joblib.load(train_path)
    test_path = pathlib['data']['test']
    X_test, y_test = joblib.load(test_path)
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(pathlib)
X_train.head()

In [ ]:
# def load_model(pathlib) from pathlib file
def load_model(pathlib, model_name='RF'):
    model_path = pathlib['models'][model_name]
    model = joblib.load(model_path)
    return model

## test load_model function

model_names = ['RF','logreg']

for model in model_names:
    print(model)
    model = load_model(pathlib, model_name=model)
    display(model)

In [ ]:
## def get_explainer()
def get_explainer(_model_pipe, X_train, labels):
    X_train_sc = _model_pipe[0].transform(X_train)
    feature_names = _model_pipe[0].get_feature_names_out()
    explainer = LimeTabularExplainer(
                    X_train_sc,
                    mode='classification',
                    feature_names=feature_names,
                    class_names=labels,
                    random_state=42
                    )
    return explainer

## test get_explainer()
explainer = get_explainer(model, X_train, labels=None)
explainer

In [ ]:
## def explain_instance()
def explain_instance(_explainer, _model_pipe, instance_to_explain):
    instance_to_explain_sc = _model_pipe[0].transform(instance_to_explain)
    explanation = _explainer.explain_instance(instance_to_explain_sc[0],
                                             _model_pipe[-1].predict_proba
                                             )
    return explanation

## test explain_instance()
explanation = explain_instance(explainer, model, X_test.head(1))
display(explanation)
explanation.as_pyplot_figure();